# Project #1
#### Team Members: Arturo White, Conor Martin , Dean Thoms, Jackie Siegel, Nate Smith
* Hypothesis:
    - Teams perform better after a bye week
        ○ The offense scores more points after a bye week
        ○ The defense allows fewer points after a bye week

    - Null hypothesis:
        ○ Bye weeks have no impact on a team's performance


In [505]:
## Import all dependencies
import pandas as pd
import numpy as np
import stat as st
import matplotlib.pyplot as plt


### Data Preparation:
    1. Perform validations on each data source (i.e., Review team names to ensure consistency and review date format)
        a. Kaggle (multiple)
        b. CFB Stats (i.e., wins / losses / points)
        c. Odd Shark (i.e., bye week data)
    2. Join Kaggle tables together
        a. Keys are going to verify per table
        b. Refer to "RELEASE.txt" file for notes on columns
            i. The "Code" columns allow you to tie the files together
    3. Join Kaggle and CFB tables using a join key of:
        a. Team Name
        b. Date
    4. Add Odd Shark data by joining on Date and Team Name; however, we want to populate a new column (i.e., "Bye Week (Yes or No)" in the data frame



In [506]:
## Import consolidated Kaggle Game Stats files
teamcodemaster = pd.read_csv('teamcodemaster.csv')
kag09 = pd.read_csv('kag09')
kag10 = pd.read_csv('kag10')
kag11 = pd.read_csv('kag11')
kag12 = pd.read_csv('kag12')
kag13 = pd.read_csv('kag13')

# Drop unnecessary "Unnamed Columns"
kag09 = kag09.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 0'], axis=1)
kag10 = kag10.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 0'], axis=1)
kag11 = kag11.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 0'], axis=1)
kag12 = kag12.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 0'], axis=1)
kag13 = kag13.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 0'], axis=1)

# Convert "Game Code" columns to integer
kag09['Game Code'] = kag09['Game Code'].astype('float').round(0)
kag10['Game Code'] = kag10['Game Code'].astype('float').round(0)
kag11['Game Code'] = kag11['Game Code'].astype('float').round(0)
kag12['Game Code'] = kag12['Game Code'].astype('float').round(0)
kag13['Game Code'] = kag13['Game Code'].astype('float').round(0)

## Data quality checks
#kaggle_df.count()
# Display column names
#for col in kaggle_df.columns: 
    #print(col) 

In [507]:
# Bring in all Game date data
game_date09 = pd.read_csv('Game Dates_2009.csv')
game_date10 = pd.read_csv('game date_2010.csv')
game_date11 = pd.read_csv('game date_2011.csv')
game_date12 = pd.read_csv('game date_2012.csv')
game_date13 = pd.read_csv('Game Dates_2013.csv')

# Merge all game date data
#game_date_list_of_dfs = [game_date09, game_date10, game_date11, game_date12, game_date13]
#game_date_df = pd.concat(game_date_list_of_dfs)
#game_date_df.head()

# Convert date from string to date type format
game_date09['Date'] = pd.to_datetime(game_date09['Date'], errors = 'raise')
game_date10['Date'] = pd.to_datetime(game_date10['Date'], errors = 'raise')
game_date11['Date'] = pd.to_datetime(game_date11['Date'], errors = 'raise')
game_date12['Date'] = pd.to_datetime(game_date12['Date'], errors = 'raise')
game_date13['Date'] = pd.to_datetime(game_date13['Date'], errors = 'raise')

# Convert "Game Code" columns to integer
game_date09['Game Code'] = game_date09['Game Code'].astype('float').round(0)
game_date10['Game Code'] = game_date10['Game Code'].astype('float').round(0)
game_date11['Game Code'] = game_date11['Game Code'].astype('float').round(0)
game_date12['Game Code'] = game_date12['Game Code'].astype('float').round(0)
game_date13['Game Code'] = game_date13['Game Code'].astype('float').round(0)

#game_dates = pd.concat([game_date09, game_date10, game_date11, game_date12, game_date13])
#game_dates.count()

In [508]:
# Merge ecah of the game dates by year
kag09_conf = pd.merge(kag09, game_date09, how = 'left', on = 'Game Code')
kag10_conf = pd.merge(kag10, game_date10, how = 'left', on = 'Game Code')
kag11_conf = pd.merge(kag11, game_date11, how = 'left', on = 'Game Code')
kag12_conf = pd.merge(kag12, game_date12, how = 'left', on = 'Game Code')
kag13_conf = pd.merge(kag13, game_date13, how = 'left', on = 'Game Code')

# Merge all kaggle data
kaggle_list_of_dfs = [kag09_conf, kag10_conf, kag11_conf, kag12_conf, kag13_conf]
kaggle_df = pd.concat(kaggle_list_of_dfs)


In [509]:
# Merge in Conferce names to DF

## Import data
conf_name_df = pd.read_csv('conference11.csv')
conf_name_df.head()

## Merge data
kaggle_conf_df = pd.merge(kaggle_df, conf_name_df, on = 'Conference Code', how = 'left')

# Rename columns that received a _x suffix
kaggle_conf_df = kaggle_conf_df.rename(columns={'Name_x': 'Name', 'Name_y': 'Conference Name', 'Subdivision_x': 'Subdivision'})

#for col in kaggle_df.columns: 
    #print(col) 

# Data quality checks
#kaggle_conf_df.count()

In [510]:
# Bring in Odd Shark data
odd_shark = pd.read_csv('teamcodemaster____Nate Edited.csv')

# Print columns to check quality
#for col in odd_shark.columns: 
    #print(col) 


In [511]:
# Join the Odd Shark data to the Kaggle stats data
temp_combined_df_1 = pd.merge(kaggle_conf_df, odd_shark, how = 'left', on = 'Name')

# Join the game date data to the temp combined df
### This is what will need to be deleted
#temp_combined_df_2 = pd.merge(temp_combined_df_1, game_date_df, how = 'left', on = 'Game Code')
    
# Drop unnecessary columns
temp_combined_df_1 = temp_combined_df_1.drop(['Team Code_y', 'Conference Code_y'], axis=1)

# Rename columns that received a _x suffix
temp_combined_df_1 = temp_combined_df_1.rename(columns={'Team Code_x': 'Team Code', 'Conference Code_x': 'Conference Code'})

# Convert all "Unnamed: X" columns to date format
temp_combined_df_1['Unnamed: 3'] = pd.to_datetime(temp_combined_df_1['Unnamed: 3'], errors = 'raise')
temp_combined_df_1['Unnamed: 4'] = pd.to_datetime(temp_combined_df_1['Unnamed: 4'], errors = 'raise')
temp_combined_df_1['Unnamed: 5'] = pd.to_datetime(temp_combined_df_1['Unnamed: 5'], errors = 'raise')
temp_combined_df_1['Unnamed: 6'] = pd.to_datetime(temp_combined_df_1['Unnamed: 6'], errors = 'raise')
temp_combined_df_1['Unnamed: 7'] = pd.to_datetime(temp_combined_df_1['Unnamed: 7'], errors = 'raise')
temp_combined_df_1['Unnamed: 8'] = pd.to_datetime(temp_combined_df_1['Unnamed: 8'], errors = 'raise')
temp_combined_df_1['Unnamed: 9'] = pd.to_datetime(temp_combined_df_1['Unnamed: 9'], errors = 'raise')
temp_combined_df_1['Unnamed: 10'] = pd.to_datetime(temp_combined_df_1['Unnamed: 10'], errors = 'raise')
temp_combined_df_1['Unnamed: 11'] = pd.to_datetime(temp_combined_df_1['Unnamed: 11'], errors = 'raise')
temp_combined_df_1['Unnamed: 12'] = pd.to_datetime(temp_combined_df_1['Unnamed: 12'], errors = 'raise')

# Add empty column 'bye' to store if the game was after a bye week or not
temp_combined_df_1['bye'] = np.nan

# Print all column names of temp_combined_df_1
#for col in temp_combined_df_2.columns: 
    #print(col) 
    
# Data quality checks
#temp_combined_df_1.count()


In [512]:
# Populate 'bye' column with an x if the game was after a bye week
## Create a loop that goes through the various bye week columns
for row in range(0, len(temp_combined_df_1)):
    if temp_combined_df_1.loc[row,'Date'] == temp_combined_df_1.loc[row, 'Unnamed: 3']:
        temp_combined_df_1.loc[row,'bye'] = 'x'
    if temp_combined_df_1.loc[row,'Date'] == temp_combined_df_1.loc[row, 'Unnamed: 4']:
        temp_combined_df_1.loc[row,'bye'] = 'x'
    if temp_combined_df_1.loc[row,'Date'] == temp_combined_df_1.loc[row, 'Unnamed: 5']:
        temp_combined_df_1.loc[row,'bye'] = 'x'
    if temp_combined_df_1.loc[row,'Date'] == temp_combined_df_1.loc[row, 'Unnamed: 6']:
        temp_combined_df_1.loc[row,'bye'] = 'x'
    if temp_combined_df_1.loc[row,'Date'] == temp_combined_df_1.loc[row, 'Unnamed: 7']:
        temp_combined_df_1.loc[row,'bye'] = 'x'
    if temp_combined_df_1.loc[row,'Date'] == temp_combined_df_1.loc[row, 'Unnamed: 8']:
        temp_combined_df_1.loc[row,'bye'] = 'x'
    if temp_combined_df_1.loc[row,'Date'] == temp_combined_df_1.loc[row, 'Unnamed: 9']:
        temp_combined_df_1.loc[row,'bye'] = 'x'
    if temp_combined_df_1.loc[row,'Date'] == temp_combined_df_1.loc[row, 'Unnamed: 10']:
        temp_combined_df_1.loc[row,'bye'] = 'x'
    if temp_combined_df_1.loc[row,'Date'] == temp_combined_df_1.loc[row, 'Unnamed: 11']:
        temp_combined_df_1.loc[row,'bye'] = 'x'
    if temp_combined_df_1.loc[row,'Date'] == temp_combined_df_1.loc[row, 'Unnamed: 12']:
        temp_combined_df_1.loc[row,'bye'] = 'x'

# Count the total number of games after the bye week
#temp_combined_df_2[temp_combined_df_2.bye == 'x'].count() 

In [513]:
# Drop the "Unnamed Columns" from the temp data frame
## We no longer need these now that we have successfully populated the 'bye' week column
temp_clean_df = temp_combined_df_1.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'], axis=1)

# Convert date from string to date type format
temp_clean_df['New Date'] = pd.to_datetime(temp_clean_df['Date'])
    
# Data quality checks
# temp_clean_df.count()

In [514]:
# Import the points for, points against, and win vs. loss information from CFB Stats

# 2009 Data
wl2009 = pd.read_csv('WL2009 Clean.csv')
wl2009 = wl2009.rename(columns={'Pts': 'Winner Points', 'Pts.1': 'Loser Points'})
wl2009['New Date'] = pd.to_datetime(wl2009['Date'])
wl2009 = wl2009.drop_duplicates(['Date', 'Winner', 'Winner Points', 'Loser', 'Loser Points'])

# 2010 Data
wl2010 = pd.read_excel('WL2010Clean.xlsx')
wl2010 = wl2010.rename(columns={'Pts': 'Winner Points', 'Pts.1': 'Loser Points'})
wl2010['New Date'] = pd.to_datetime(wl2010['Date'])
wl2010 = wl2010.drop_duplicates(['Date', 'Winner', 'Winner Points', 'Loser', 'Loser Points'])

# 2011 Data
wl2011 = pd.read_excel('WL2011_Cleaned_edited7_27_19.xlsx')
wl2011 = wl2011.rename(columns={'Pts': 'Winner Points', 'Pts.1': 'Loser Points'})
wl2011['New Date'] = pd.to_datetime(wl2011['Date'])
wl2011 = wl2011.drop_duplicates(['Date', 'Winner', 'Winner Points', 'Loser', 'Loser Points'])

# 2012 Data
wl2012 = pd.read_csv('WL2012CLEAN.csv')
wl2012 = wl2012.rename(columns={'Pts': 'Winner Points', 'Pts.1': 'Loser Points'})
wl2012['New Date'] = pd.to_datetime(wl2012['Date'])
wl2012 = wl2012.drop_duplicates(['Date', 'Winner', 'Winner Points', 'Loser', 'Loser Points'])

# 2013 Data
wl2013 = pd.read_csv('WL2013CLEAN.csv')
wl2013 = wl2013.rename(columns={'Pts': 'Winner Points', 'Pts.1': 'Loser Points'})
wl2013['New Date'] = pd.to_datetime(wl2013['Date'])
wl2013 = wl2013.drop_duplicates(['Date', 'Winner', 'Winner Points', 'Loser', 'Loser Points'])

# Merge all W/L and Points data
points_list_of_dfs = [wl2009, wl2010, wl2011, wl2012, wl2013]
point_df = pd.concat(points_list_of_dfs, sort = 'True')

# Data quality tests
#point_df.count()

### Data Munging:
    1. Check for any NaN values or null records in critical fields
    2. Remove any unnecessary teams from the data frame
    3. Disaggregate data frame into two sub components
        a. Create data frame that only contains the game information after bye week
            i. EX: bye_week_df = df[df['Bye Week (Yes or No)'] == "Yes"]Create data frame that contains all but the games after bye week
            ii. EX: bye_week_df = df[df['Bye Week (Yes or No)'] == "No"]
    4. Separate dataframes by year (**Note this isn't 100% necessary as we could always group-by year and team name later, but I think separating the dataframes will make it easier on us)

### Analysis Prep:
    1. Group each df by team name
    2. Calculate the mean of each critical field for non-bye week dataframes (i.e., mean of total points, off. Points, etc.)
        a. Note I will be referring to these data frames as "Grouped by Team Non-Bye Week df" for the remainder of this pseudocode
    3. Join in the "Bye Week Stats df" (i.e., the bye week data frame grouped by team name) into the "Grouped by Team Non-Bye Week df"
        a. Note that the "Grouped by Team Non-Bye Week df" should be the left portion of the join
        b. Add prefix to all right side of join columns to easily distinguish that those are the by week stats
    4. Remove unnecessary columns for ease of workability (i.e., create offense only df, defense only df, total points df, etc.)



In [515]:
# Merge the points for, points against, and win vs. loss information from CFB Stats into temp data frame

## Merge based on the winner
consolidated_df_v0 = pd.merge(temp_clean_df, point_df, how = 'left', left_on = ['Name', 'New Date'], right_on = ['Winner', 'New Date'])
# Rename _x suffix
consolidated_df_v0 = consolidated_df_v0.rename(columns={'Date_x': 'Date'})
# Drop unnecessary repeated columns (i.e., "Date_y")
consolidated_df_v0 = consolidated_df_v0.drop(['Date_y'], axis=1)

## Merge based on the loser
consolidated_df = pd.merge(consolidated_df_v0, point_df, how = 'left', left_on = ['Name', 'New Date'], right_on = ['Loser', 'New Date'])
# Rename _x suffix
consolidated_df = consolidated_df.rename(columns={'Date_x': 'Date'})
# Drop unnecessary repeated columns (i.e., "Date_y")
consolidated_df = consolidated_df.drop(['Date_y'], axis=1)

# Data quality checks
#consolidated_df.count()

# Print all column names of consolidated_df
#for col in consolidated_df.columns: 
    #print(col) 



In [516]:
# Identify if record was a Win or Loss

## add outcome column to store win / loss
consolidated_df['outcome'] = ''
## Loop through data set and add win / loss to each record
for row in range(0, len(consolidated_df)):
    if consolidated_df.loc[row,'Winner_x'] == consolidated_df.loc[row,'Name']:
        consolidated_df.loc[row,'outcome'] = 'Win'
    else:
        consolidated_df.loc[row,'outcome'] = 'Loss'

In [517]:
# Identify and implement a points for and points against column

## add points for and against columns to store data
consolidated_df['points_for'] = ''
consolidated_df['points_against'] = ''

## Loop through data set and add win / loss to each record
for row in range(0, len(consolidated_df)):
    if consolidated_df.loc[row,'outcome'] == 'Win':
        consolidated_df.loc[row,'points_for'] = consolidated_df.loc[row,'Winner Points_x']
    else:
        consolidated_df.loc[row,'points_for'] = consolidated_df.loc[row,'Loser Points_y']
    if consolidated_df.loc[row,'outcome'] == 'Win':
        consolidated_df.loc[row,'points_against'] = consolidated_df.loc[row,'Loser Points_x']
    else:
        consolidated_df.loc[row,'points_against'] = consolidated_df.loc[row,'Winner Points_y']


In [518]:
# Drop all unnecessary columns from consolidated df

# Rename _x suffix
#consolidated_df = consolidated_df.rename(columns={'Date_x': 'Date'})
drop_columns = ['Day_x', 'Winner_x', 'Winner Points_x', 'Loser_x', 'Loser Points_x', 'Day_y', 'Winner_y', 'Winner Points_y', 'Loser_y', 'Loser Points_y']
# Drop unnecessary repeated columns (i.e., "Date_y")
consolidated_df = consolidated_df.drop(drop_columns, axis=1)


In [519]:
consolidated_df.to_csv('df.csv')
consolidated_df.groupby('bye').count()

,Team Code,Game Code,Rush Att,Rush Yard,Rush TD,Pass Att,Pass Comp,Pass Yard,Pass TD,Pass Int,...,Stadium Code,Site,Conference Name,Subdivision,New Date,Wk_x,Wk_y,outcome,points_for,points_against
bye,,,,,,,,,,,,,,,,,,,,,
x,404,404,404,404,404,404,404,404,404,404,...,404,404,404,404,404,22,26,404,379,379


### Analysis / Statistical Results:
    1. Perform ANOVA analysis on respective fields from mean season vs. after bye week
        a. Determine if there is significance between the data
    2. T-test to see if there is statistical significance b/n post bye week performance compared to avg. performance
    


### Visualization:
    1. Box and Whisker plot of statistical results
    2. Create clustered bar chart for each of the major comparisons:
        a. Win % over the five years (each cluster of bars is a team)
        b. Avg Off. Pts. Over the five years (each cluster of bars is a team)
        c. Avg Def. Pts Allowed Over the five years (each cluster of bars is a team)